We will create auto generated captions for youtube videos.

Tasks:
- create a function to return the plain text captions from a video link
- collect all of the desired YT links
- generate and store captions for all desired videos

In [ ]:
#function i ripped straight from chatGPT
def get_auto_captions(video_url, lang='en'):
    ydl_opts = {
        'skip_download': True,  # Don't download the video
        'writesubtitles': True,
        'writeautomaticsub': True,  # Auto-generated subs
        'subtitleslangs': [lang],
        'outtmpl': '-',  # No file output
        'quiet': True,
    }

    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(video_url, download=False)  # Extract metadata only
        subtitles = info.get('subtitles') or info.get('automatic_captions')
        
        if not subtitles or lang not in subtitles:
            print("No subtitles found.")
            return None
        
        # Fetch the first subtitle URL
        subtitle_url = subtitles[lang][0]['url']
        
        # Download subtitle data
        response = requests.get(subtitle_url)
        
        if response.status_code != 200:
            return response.text  # Returns the caption content as text
        try:
            data = json.loads(response.text)
        except json.JSONDecodeError:
            print("Failed to parse subtitle JSON.")
            return None

        # Extract all UTF-8 text snippets
        replacements = {
            "battalion": " Bataleon",
            "rosal": " Rossignol",
            "ganu": " GNU",
            "baru": " Biru"
        }

        lines = []
        for event in data.get('events', []):
            if 'segs' in event:
                line = ''
                for seg in event['segs']:
                    word = seg.get('utf8', '')
                    lower_word = word.lower().strip()
                    replaced_word = replacements.get(lower_word, word)
                    line += replaced_word
                lines.append(line.strip())

        return ' '.join(lines)

In [ ]:
videos = scrapetube.get_channel("UCgHN8SXc6GXm8-Rx0pjWGIw", limit = 1000) #channel ID
main_link = "https://www.youtube.com/watch?v="
video_captions = []
for video in videos:
    caption = get_auto_captions(main_link + video['videoId'])
    if caption is not None:
        video_captions.append(video['title']['runs'][0]['text'] + ": " + caption)